<a href="https://colab.research.google.com/github/yunyoungwoo/2024S-Ajou-ML-FP/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as v2
from tqdm import tqdm
import numpy as np
import os

In [ ]:
# 데이터프레임을 사용하여 커스텀 데이터셋 생성
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]  # 이미지 파일 경로
        image = Image.open(image_path).convert('RGB')  # 이미지 열기
        label = self.dataframe.iloc[idx, 1]  # 레이블

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# 데이터프레임 로드 및 데이터 전처리 파이프라인 정의
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # torch 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
    transforms.RandomRotation(degrees=90),  # 랜덤 회전
    transforms.RandomHorizontalFlip(p=0.5)  # 랜덤 수평 뒤집기
])

In [ ]:
# 데이터프레임을 사용하여 데이터셋 생성
train_dataset = BaseDataset(train_df, transform=train_transforms)
val_dataset = BaseDataset(val_df, transform=train_transforms)
test_dataset = BaseDataset(test_df, transform=train_transforms)

In [ ]:
# 데이터로더 생성
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
import torch.nn.functional as F

class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=512 * 7 * 7, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=10)  # 예시로 10개의 클래스에 대한 출력을 가정합니다.
        self.dropout = nn.Dropout(p=0.5)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.batchnorm5 = nn.BatchNorm2d(256)
        self.batchnorm6 = nn.BatchNorm2d(256)
        self.batchnorm7 = nn.BatchNorm2d(512)
        self.batchnorm8 = nn.BatchNorm2d(512)

    def forward(self, x):
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))
        x = self.pool(F.relu(self.batchnorm4(self.conv4(x))))
        x = self.pool(F.relu(self.batchnorm5(self.conv5(x))))
        x = self.pool(F.relu(self.batchnorm6(self.conv6(x))))
        x = self.pool(F.relu(self.batchnorm7(self.conv7(x))))
        x = self.pool(F.relu(self.batchnorm8(self.conv8(x))))
        x = x.view(-1, 512 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
# 모델 생성 및 CUDA 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedCNN().to(device)

In [ ]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습률 스케줄러 정의
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
# 학습 함수 정의
def train(model, train_loader, optimizer, criterion, lr_scheduler):
    model.train()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()  # 스케줄러 업데이트
        _, predicted = outputs.max(1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        running_loss += loss.item()
    accuracy = accuracy_score(all_targets, all_predictions)
    macro_f1 = f1_score(all_targets, all_predictions, average='macro')
    train_loss = running_loss / len(train_loader)
    return train_loss, accuracy, macro_f1

In [ ]:
# 검증 함수 정의
def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = outputs.max(1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            running_loss += loss.item()
    accuracy = accuracy_score(all_targets, all_predictions)
    macro_f1 = f1_score(all_targets, all_predictions, average='macro')
    test_loss = running_loss / len(test_loader)
    return test_loss, accuracy, macro_f1

In [ ]:
# 학습 및 검증 실행
NUM_EPOCHS = 10
best_val_acc = 0.0
patience = 5
counter = 0
best_val_loss = np.inf

In [ ]:
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc, train_f1 = train(model, train_loader, optimizer, criterion, lr_scheduler)
    test_loss, test_acc, test_f1 = test(model, test_loader, criterion)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Acc: {train_acc:.2f}%, Train F1: {train_f1:.2f}, Test Acc: {test_acc:.2f}%, Test F1: {test_f1:.2f}")

    # 검증 손실이 이전 최고 손실값보다 낮은 경우 모델 저장
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        torch.save(model.state_dict(), 'best_model.pth')
        counter = 0  # 성능이 향상되었으므로 카운터 초기화
    else:
        counter += 1  # 성능이 향상되지 않았으므로 카운터 증가

    # 카운터가 지정된 인내심(patience)을 초과하면 학습 종료
    if counter >= patience:
        print(f"Early stopping at epoch {epoch+1} as test loss did not improve for {patience} epochs.")
        break

NameError: name 'train' is not defined